In [3]:
# 필요한 라이브러리 import (통합)
import pandas as pd
import numpy as np
import time
import psutil
from datetime import datetime
from pulp import LpProblem, LpVariable, LpInteger, LpMaximize, lpSum, PULP_CBC_CMD

print("✅ 모든 라이브러리 import 완료")

✅ 모든 라이브러리 import 완료


In [4]:
# 1. 더미 데이터 생성 및 저장
np.random.seed(42)
num_skus = 20
num_stores = 100

colors = ['black', 'gray', 'white', 'navy', 'red', 'green', 'blue', 'yellow']
sizes = ['S', 'M', 'L', 'XS', 'XL', 'XXL', 'XXS']

sku_list = []
for i in range(num_skus):
    sku_list.append({
        'sku_id': f'SKU_{i+1}',
        'color': np.random.choice(colors),
        'size': np.random.choice(sizes),
        'supply': np.random.randint(50, 200)
    })
df_skus = pd.DataFrame(sku_list)
df_skus.to_csv('data/sku.csv', index=False)

store_list = []
for j in range(num_stores):
    store_list.append({
        'store_id': f'ST_{j+1}',
        'cap': np.random.randint(100, 500)
    })
df_stores = pd.DataFrame(store_list)
df_stores.to_csv('data/store.csv', index=False)

demand_rows = []
for _, sku in df_skus.iterrows():
    for _, store in df_stores.iterrows():
        demand_rows.append({
            'sku_id': sku['sku_id'],
            'store_id': store['store_id'],
            'demand': np.random.randint(0, store['cap'] // 5)
        })
df_demand = pd.DataFrame(demand_rows)
df_demand.to_csv('data/demand.csv', index=False)

In [5]:
# 최적화 라이브러리 import
from pulp import LpProblem, LpVariable, LpInteger, LpMaximize, lpSum, PULP_CBC_CMD

# 데이터 불러오기
skus = pd.read_csv('data/sku.csv')
stores = pd.read_csv('data/store.csv')
demand = pd.read_csv('data/demand.csv')

# 집합 정의
C_color = skus[~skus['color'].isin(['black','gray','white','navy'])]['sku_id'].tolist()
S_size  = skus[~skus['size'].isin(['S','M','L'])]['sku_id'].tolist()

# 글로벌 비율 계산 (demand 기반)
merged = demand.merge(stores, on='store_id').merge(skus[['sku_id','color','size']], on='sku_id')

r_color_max = merged[~merged['color'].isin(['black','gray','white','navy'])]['demand'].sum() / merged['demand'].sum()
r_color_min = 0.1 # 도메인 전문가 지정

r_size_max  = merged[~merged['size'].isin(['S','M','L'])]['demand'].sum() / merged['demand'].sum()
r_size_min  = 0.05 # 도메인 전문가 지정

print(f"Color ratio range: {r_color_min:.2f} - {r_color_max:.2f}")
print(f"Size ratio range: {r_size_min:.2f} - {r_size_max:.2f}")

Color ratio range: 0.10 - 0.51
Size ratio range: 0.05 - 0.55


In [6]:
# 문제 정의
prob = LpProblem("SKU_Distribution", LpMaximize)
x = LpVariable.dicts("x", (skus['sku_id'], stores['store_id']), lowBound=0, cat=LpInteger)

# 목적함수
prob += lpSum(x[i][j] for i in skus['sku_id'] for j in stores['store_id'])

# 제약조건
# 1. 각 SKU의 공급량 제약
for _, sku in skus.iterrows():
    prob += lpSum(x[sku['sku_id']][j] for j in stores['store_id']) <= sku['supply']

# 2. 각 store-sku 조합의 수요량 제약
# for _, row in demand.iterrows():
#     prob += x[row['sku_id']][row['store_id']] <= row['demand']

# 3. 각 store의 용량 및 비율 제약
for _, store in stores.iterrows():
    j = store['store_id']
    all_alloc   = lpSum(x[i][j] for i in skus['sku_id'])
    color_alloc = lpSum(x[i][j] for i in C_color)
    size_alloc  = lpSum(x[i][j] for i in S_size)
    
    prob += all_alloc <= store['cap']
    prob += color_alloc >= r_color_min * all_alloc
    prob += color_alloc <= r_color_max * all_alloc
    prob += size_alloc  >= r_size_min * all_alloc
    prob += size_alloc  <= r_size_max * all_alloc

---

In [7]:
# 시스템 정보 및 최적화 설정 분석

import os
import psutil
import time
from datetime import datetime

print("=== 시스템 정보 분석 ===")

# 1. CPU 정보 확인
logical_cores = psutil.cpu_count(logical=True)   # 논리 코어 (하이퍼스레딩 포함)
physical_cores = psutil.cpu_count(logical=False) # 물리 코어
print(f"물리 CPU 코어 수: {physical_cores}")
print(f"논리 CPU 코어 수: {logical_cores}")
print(f"권장 스레드 수: {logical_cores - 1} (시스템 1개 코어 여유)")

# 2. 메모리 정보
memory = psutil.virtual_memory()
print(f"총 메모리: {memory.total / (1024**3):.1f} GB")
print(f"사용 가능 메모리: {memory.available / (1024**3):.1f} GB")
print(f"메모리 사용률: {memory.percent}%")

print(f"\n=== 최적 스레드 설정 권장사항 ===")
print(f"• 소규모 문제: threads=1~2")
print(f"• 중간 문제: threads={min(4, logical_cores-1)}")
print(f"• 대규모 문제: threads={logical_cores-1}")
print(f"• 최대 성능: threads={logical_cores}")

print(f"\n=== PuLP/CBC 기본값 ===")
print(f"• threads 기본값: 1 (단일 스레드)")
print(f"• timeLimit 기본값: 무제한")
print(f"• fracGap 기본값: 0.0 (완벽한 해)")
print(f"• presolve 기본값: True (활성화)")


=== 시스템 정보 분석 ===
물리 CPU 코어 수: 8
논리 CPU 코어 수: 16
권장 스레드 수: 15 (시스템 1개 코어 여유)
총 메모리: 31.8 GB
사용 가능 메모리: 10.4 GB
메모리 사용률: 67.4%

=== 최적 스레드 설정 권장사항 ===
• 소규모 문제: threads=1~2
• 중간 문제: threads=4
• 대규모 문제: threads=15
• 최대 성능: threads=16

=== PuLP/CBC 기본값 ===
• threads 기본값: 1 (단일 스레드)
• timeLimit 기본값: 무제한
• fracGap 기본값: 0.0 (완벽한 해)
• presolve 기본값: True (활성화)


In [8]:
# PRESOLVE란 무엇인가?

print("=== PRESOLVE (전처리) 설명 ===\n")

print("🔧 PRESOLVE가 하는 일:")
print("1. 불필요한 변수 제거")
print("   - 값이 0으로 고정된 변수들")
print("   - 실제로 영향을 주지 않는 변수들")
print("\n2. 중복 제약조건 제거")
print("   - 동일한 의미의 제약조건들")
print("   - 이미 다른 제약에 포함된 조건들")
print("\n3. 변수 범위 축소")
print("   - 상한/하한 더 정확하게 계산")
print("   - 정수 변수의 가능 범위 줄이기")
print("\n4. 문제 단순화")
print("   - 선형 변환으로 문제 크기 줄이기")
print("   - 계수 정규화")

print("\n📊 PRESOLVE 효과:")
print("• 장점: 문제 크기 ↓, 해결 시간 ↓, 메모리 사용량 ↓")
print("• 단점: 전처리 시간 추가 소요")
print("• 권장: 대부분의 경우 사용 (특히 큰 문제)")

print("\n⚠️ PRESOLVE를 끄는 경우:")
print("• 매우 작은 문제 (변수 < 100개)")
print("• 이미 최적화된 모델")
print("• 디버깅할 때 (원본 문제 구조 유지)")

print("\n💡 설정 방법:")
print("presolve=True   # 전처리 사용 (권장)")
print("presolve=False  # 전처리 사용 안함")


=== PRESOLVE (전처리) 설명 ===

🔧 PRESOLVE가 하는 일:
1. 불필요한 변수 제거
   - 값이 0으로 고정된 변수들
   - 실제로 영향을 주지 않는 변수들

2. 중복 제약조건 제거
   - 동일한 의미의 제약조건들
   - 이미 다른 제약에 포함된 조건들

3. 변수 범위 축소
   - 상한/하한 더 정확하게 계산
   - 정수 변수의 가능 범위 줄이기

4. 문제 단순화
   - 선형 변환으로 문제 크기 줄이기
   - 계수 정규화

📊 PRESOLVE 효과:
• 장점: 문제 크기 ↓, 해결 시간 ↓, 메모리 사용량 ↓
• 단점: 전처리 시간 추가 소요
• 권장: 대부분의 경우 사용 (특히 큰 문제)

⚠️ PRESOLVE를 끄는 경우:
• 매우 작은 문제 (변수 < 100개)
• 이미 최적화된 모델
• 디버깅할 때 (원본 문제 구조 유지)

💡 설정 방법:
presolve=True   # 전처리 사용 (권장)
presolve=False  # 전처리 사용 안함


In [9]:
# 최적화 진행 상황 모니터링

print("=== 최적화 진행 상황 추적 방법 ===\n")

def solve_with_monitoring(prob, solver_options=None):
    """
    최적화를 실행하면서 진행 상황을 모니터링
    """
    if solver_options is None:
        solver_options = {}
    
    print(f"🚀 최적화 시작: {datetime.now().strftime('%H:%M:%S')}")
    start_time = time.time()
    
    # 기본 솔버 옵션 설정
    default_options = {
        'msg': True,
        'logPath': 'optimization.log',  # 로그 파일 생성
        'timeLimit': 300,  # 10분 제한
        'threads': min(4, psutil.cpu_count()-1)
    }
    default_options.update(solver_options)
    
    print(f"⚙️  솔버 설정:")
    for key, value in default_options.items():
        print(f"   {key}: {value}")
    
    # 최적화 실행
    try:
        solution_status = prob.solve(PULP_CBC_CMD(**default_options))
        end_time = time.time()
        elapsed_time = end_time - start_time
        
        print(f"\n⏱️  총 소요 시간: {elapsed_time:.2f}초")
        print(f"🏁 완료 시각: {datetime.now().strftime('%H:%M:%S')}")
        
        return solution_status, elapsed_time
    
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None, None

print("💡 진행 상황 파악 방법:")
print("1. msg=True로 콘솔 출력 확인")
print("2. logPath로 상세 로그 파일 생성")
print("3. timeLimit으로 최대 시간 제한")
print("4. 중간에 Ctrl+C로 중단 가능")

print("\n📋 로그 파일에서 확인 가능한 정보:")
print("• Gap: 현재해와 최적해의 차이 (%)")
print("• Objective: 현재 목적함수 값")
print("• Nodes: 탐색한 노드 수")
print("• Time: 경과 시간")

print("\n🔍 실시간 모니터링 예시:")
print("Welcome to the CBC MILP Solver")
print("Version: 2.10.3")
print("...")
print("Cgl0004I processed model has 1000 rows, 2000 columns")
print("Coin0506I Presolve 50 (-950) rows, 500 (-1500) columns") 
print("...")
print("Cbc0012I Integer solution of 12345 found by heuristic")
print("Cbc0001I Search completed - best objective 12345")
print("Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost")


=== 최적화 진행 상황 추적 방법 ===

💡 진행 상황 파악 방법:
1. msg=True로 콘솔 출력 확인
2. logPath로 상세 로그 파일 생성
3. timeLimit으로 최대 시간 제한
4. 중간에 Ctrl+C로 중단 가능

📋 로그 파일에서 확인 가능한 정보:
• Gap: 현재해와 최적해의 차이 (%)
• Objective: 현재 목적함수 값
• Nodes: 탐색한 노드 수
• Time: 경과 시간

🔍 실시간 모니터링 예시:
Welcome to the CBC MILP Solver
Version: 2.10.3
...
Cgl0004I processed model has 1000 rows, 2000 columns
Coin0506I Presolve 50 (-950) rows, 500 (-1500) columns
...
Cbc0012I Integer solution of 12345 found by heuristic
Cbc0001I Search completed - best objective 12345
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost


In [ ]:
# 🚀 최대 스레드로 최적화 실행
print("=== 최대 스레드 최적화 실행 ===")
print(f"📊 문제 규모: {len(skus)} SKUs × {len(stores)} Stores = {len(skus) * len(stores):,} 변수")

# 시스템 정보 확인
logical_cores = psutil.cpu_count(logical=True)
physical_cores = psutil.cpu_count(logical=False)
print(f"💻 시스템 정보: 물리 코어 {physical_cores}개, 논리 코어 {logical_cores}개")

# logPath 없이 최대 스레드로 실행
print(f"🚀 최적화 시작: {datetime.now().strftime('%H:%M:%S')}")
start_time = time.time()

# 최대 스레드 설정
max_threads = logical_cores  # 모든 논리 코어 사용
solver_options = {
    'msg': True,              # 실시간 콘솔 출력
    'timeLimit': 300,         # 5분 제한  
    'threads': max_threads    # 최대 스레드 사용
}

print(f"⚙️  솔버 설정:")
for key, value in solver_options.items():
    print(f"   {key}: {value}")

print(f"\n🔥 최대 성능으로 최적화 시작! (스레드: {max_threads}개)")

# 최적화 실행
try:
    solution_status = prob.solve(PULP_CBC_CMD(**solver_options))
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"\n⏱️  총 소요 시간: {elapsed_time:.2f}초")
    print(f"🏁 완료 시각: {datetime.now().strftime('%H:%M:%S')}")
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    solution_status = None
    elapsed_time = None


=== 최대 스레드 최적화 실행 ===
📊 문제 규모: 20 SKUs × 100 Stores = 2,000 변수
💻 시스템 정보: 물리 코어 8개, 논리 코어 16개
🚀 최적화 시작: 17:37:46
⚙️  솔버 설정:
   msg: True
   timeLimit: 300
   threads: 16

🔥 최대 성능으로 최적화 시작! (스레드: 16개)


In [7]:
# 결과 분석 및 출력
status_dict = {
    1: "최적해 발견",
    0: "시간 제한으로 중단", 
    -1: "실행 불가능한 문제",
    -2: "무한대 해",
    -3: "정의되지 않음"
}

print(f"📊 최적화 결과:")
if 'solution_status' in locals():
    print(f"상태: {status_dict.get(solution_status, '알 수 없음')} (코드: {solution_status})")
    
    if solution_status == 1:
        # 성공적인 해결
        objective_value = prob.objective.value()
        print(f"목적함수 값: {objective_value:.0f}")
        
        # 결과 데이터 수집
        result_data = []
        for i in skus['sku_id']:
            for j in stores['store_id']:
                value = x[i][j].value()
                if value and value > 0:
                    result_data.append({
                        'sku_id': i,
                        'store_id': j,
                        'allocation': int(value)
                    })
        
        if result_data:
            result_df = pd.DataFrame(result_data)
            
            print(f"\n📈 결과 요약:")
            print(f"총 할당량: {result_df['allocation'].sum():,}")
            print(f"할당된 조합: {len(result_df):,}개")
            print(f"평균 할당량: {result_df['allocation'].mean():.1f}")
            
            # 상위 결과 출력
            print(f"\n🔝 할당량 상위 10개:")
            top_results = result_df.nlargest(10, 'allocation')
            print(top_results)
            
            # 결과 저장
            result_df.to_csv('data/allocation_result.csv', index=False)
            print(f"\n💾 결과 저장: data/allocation_result.csv")
            
        else:
            print("할당된 결과가 없습니다.")
            
    elif solution_status == 0:
        print("💡 시간 제한으로 중단되었습니다. timeLimit을 늘려보세요.")
        
    else:
        print("💡 문제를 해결할 수 없습니다. 제약조건을 확인해보세요.")
else:
    print("아직 최적화가 실행되지 않았습니다.")


📊 최적화 결과:
아직 최적화가 실행되지 않았습니다.


## 📚 참고자료 및 추가 분석

In [ ]:
# 추가 분석: 결과가 있을 때만 실행
if 'result_df' in locals() and not result_df.empty:
    print("📊 상세 분석:")
    
    # SKU별 할당 현황
    sku_summary = result_df.groupby('sku_id')['allocation'].agg(['sum', 'count', 'mean']).round(1)
    sku_summary.columns = ['총할당량', '할당상점수', '평균할당량']
    print(f"\n📦 SKU별 할당 현황 (상위 5개):")
    print(sku_summary.nlargest(5, '총할당량'))
    
    # 상점별 할당 현황
    store_summary = result_df.groupby('store_id')['allocation'].agg(['sum', 'count']).round(1)
    store_summary.columns = ['총할당량', '할당SKU수']
    print(f"\n🏪 상점별 할당 현황 (상위 5개):")
    print(store_summary.nlargest(5, '총할당량'))
    
    print(f"\n💡 CBC 솔버 매개변수 참고:")
    print("지원 매개변수: msg, timeLimit, threads, logPath")
    print("미지원: fracGap (자동처리), presolve (자동활성화), cuts (자동활성화)")
    
    print(f"\n✅ 분석 완료!")
else:
    print("📋 최적화가 성공적으로 완료되지 않아 상세 분석을 생략합니다.")
    print("\n💡 CBC 솔버 매개변수 참고:")
    print("• msg=True: 실시간 진행상황 출력")  
    print("• timeLimit=300: 시간 제한 (초)")
    print("• threads=4: 병렬 처리 스레드 수")
    print("• logPath='log.txt': 로그 파일 저장 (msg 출력 비활성화됨)")